### Start Tensof Flow Serving docker with our saved model as input

In [11]:
import subprocess
cmd = '''docker run --rm
         -p 8500:8500                                                         
         -v /tmp/tf_tutorial/model:/tmp/model                                             
         yesuprelease/tensorflow-serving                                      
         bazel-bin/tensorflow_serving/model_servers/tensorflow_model_server   
         --model_base_path='/tmp/model'                                       
         --model_name='tf_demo'
      '''
# Run our docker
p = subprocess.Popen(cmd.replace('\n',''), shell=True)

### Get sample data from big query 

In [3]:

s=r'''SELECT regexp_extract(sample_path, '.*(\\..+)$') as suffix,
             content
      FROM  [bigquery-public-data:github_repos.sample_contents]
      WHERE RAND() < 1/10
          AND content IS NOT NULL
          AND content != ''
          and length(content) > 1024
      HAVING suffix IS NOT NULL             
             AND suffix in ('.py','.c','.rb')
      LIMIT 100;'''

from gcloud import bigquery
c = bigquery.Client()
query = c.run_sync_query(s)
query.run()

### Proccess data so we can send it to our model server

In [7]:
import numpy as np 

def transform_content(content):
    content = content.encode('utf-8')
    content = content[:1024]    
    return content

code_snippets = np.transpose(np.array([transform_content(row[1]) for row in query.rows if row[0]]))
language_codes = np.array([row[0].encode('utf-8') for row in query.rows])

print code_snippets.shape

(100,)


### Import all needed libraries

In [8]:
from tensorflow_serving.apis import prediction_service_pb2
from tensorflow_serving.apis import predict_pb2

from tensorflow.python.saved_model import builder as saved_model_builder
from tensorflow.python.saved_model import signature_constants
from tensorflow.python.saved_model import tag_constants
from tensorflow.python.saved_model import signature_def_utils
from tensorflow.python.saved_model import utils

import grpc
import tensorflow as tf

### Start communication channel using grpc, get stub,  build request, and generate protobuff

In [9]:
channel = grpc.insecure_channel('localhost:8500')

stub = prediction_service_pb2.PredictionServiceStub(channel)

request = predict_pb2.PredictRequest()
request.model_spec.name = 'tf_demo'
request.model_spec.signature_name = 'predict_language'
request.inputs['content'].CopyFrom(
        tf.contrib.util.make_tensor_proto(code_snippets, shape=code_snippets.shape))

### Get Prediction

In [17]:
response = stub.Predict(request, 1)

### Translate results to logits, predicitons and accuracy

In [19]:
logits_tensor = tf.contrib.util.make_ndarray(response.outputs['logits'])
predict_tensor = tf.contrib.util.make_ndarray(response.outputs['prediction'])

language_codes_array = np.array(['UNKNOWN','.py','.c','.rb'])
predicted_laguage_codes=language_codes_array[predict_tensor]
print logits_tensor
accuracy = np.mean(predicted_laguage_codes == language_codes)
print "accuracy: {}".format(accuracy)
for predicted_language_code, actual_language_code in zip(predicted_laguage_codes, language_codes):
    print 'predicted: {}, actual: {}'.format(predicted_language_code, actual_language_code)
    


[[  0.          13.88054657   0.        ]
 [  0.           0.          12.50568199]
 [  0.           0.          26.37464905]
 [  0.           2.53996301   9.92933846]
 [  0.           1.68892777  10.62524891]
 [  0.          11.04464436   1.96848619]
 [  0.           7.18550587   5.4188714 ]
 [  0.           0.          12.55268097]
 [  0.           6.51946783   6.48295641]
 [  0.           1.95149028  10.25632286]
 [  0.           0.          29.86507034]
 [  0.           9.23156166   3.65585852]
 [  0.           0.96755266  11.06261063]
 [  0.           0.          14.2930336 ]
 [  0.           0.          30.63389587]
 [  0.          17.02863503   0.        ]
 [  0.          10.00269699   2.74343276]
 [  0.          35.70146942   0.        ]
 [  0.          16.2253952    0.        ]
 [  0.          12.14783573   0.90165758]
 [  0.           8.61195564   4.03998947]
 [  0.           9.85961342   2.88267732]
 [  0.          13.14874458   0.        ]
 [  0.          10.10594654   2.62